## SQLAlchemy

##### Author: Alex Sherman | alsherman@deloitte.com

In [ ]:
import os
from IPython.core.display import display, HTML
from configparser import ConfigParser, ExtendedInterpolation

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')
TEST_DB_PATH = config['DATABASES']['TEST_DB_PATH']
DB_PATH = config['DATABASES']['LESSON_DB_PATH']
STACKOVERFLOW_ZIP_NAME = config['DATABASES']['STACKOVERFLOW_ZIP_NAME']
STACKOVERFLOW_CSV_NAME = config['DATABASES']['STACKOVERFLOW_CSV_NAME']

SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

It provides a full suite of well known enterprise-level persistence patterns, designed for efficient and high-performing database access, adapted into a simple and Pythonic domain language.

In [ ]:
sqlalchemy_url = 'http://www.sqlalchemy.org/'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(sqlalchemy_url)
HTML(iframe)

Python's SQLAlchemy and Object-Relational Mapping

A common task when programming any web service is the construction of a solid database backend. In the past, programmers would write raw SQL statements, pass them to the database engine and parse the returned results as a normal array of records. Nowadays, programmers can write Object-relational mapping (ORM) programs to remove the necessity of writing tedious and error-prone raw SQL statements that are inflexible and hard-to-maintain.

ORM is a programming technique for converting data between incompatible type systems in object-oriented programming languages. Usually, the type system used in an OO language such as Python contains types that are non-scalar, namely that those types cannot be expressed as primitive types such as integers and strings

- Source: http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/
- Source: https://www.pythonsheets.com/notes/python-sqlalchemy.html


- create_engine: The return value of create_engine() is an instance of Engine, and it represents the core interface to the database, adapted through a dialect that handles the details of the database and DBAPI in use. In this case the SQLite dialect will interpret instructions to the Python built-in sqlite3 module.

In [ ]:
tutorial_url = 'http://docs.sqlalchemy.org/en/latest/orm/tutorial.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(tutorial_url)
HTML(iframe)

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

#### Declarative Base
Declarative base maintains a catalog of classes and tables. It keeps track of all the database tables that are accessible through SQLAlchemy.

In [ ]:
from sqlalchemy import Column, Text, Integer

#### SQLAlchemy column and data types
Each database field is provided a column and data type in SQLAlchemy

- Column: specifies a column that can read, write and store a particular type of Python data.  
- Text: In SQL, usually corresponds to CLOB or TEXT
- Integer: integer Data type

In [ ]:
# provide a class name for the database table
class Sections(Base):
    # provide a name used in SQL to query the table
    __tablename__ = 'SECTIONS'

    # provide column names and data types
    # for each field in the database
    # additional paramaters are available for the columns and data types
    section_id = Column(Integer(), primary_key=True, autoincrement=True)
    section_name = Column(Text())
    section_text = Column(Text())

    def __repr__(self):
        return '<Sections section_id: {} | section_name: {}>'.format(
            self.section_id, self.section_name)

#### SQLAlchemy Classes

Each SQLAlchemy class corresponds to a database table.

Within the class, we define details about the table to which we’ll be mapping, primarily the table name, and names and datatypes of columns.

A class using Declarative at a minimum needs a __tablename__ attribute, and at least one Column which is part of a primary key. 

Outside of what the mapping process does to our class, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

In [ ]:
sections = Sections()
sections

**_ _repr_ _** (represent) provides the output when using print. This explains why the output is sometimes different when using print  

In [ ]:
section = Sections(
      section_name = 'first section'
    , section_text = 'text from the first section'
)
section

In [ ]:
list_one = ['a','b','c']
list_two = [1,2,3]

print(zip(list_one, list_two))
zip(list_one, list_two)

In [ ]:
# confirm test database directory
TEST_DB_PATH

#### Below are common urls to connect to a variery of databases

SQLAlchemy includes dialects for common databases, including Microsoft SQL Server, MySQL, Oracle, PostgreSQL, SQLite, and more. Additional external dialects are available for other databases like Amazon Redshift.


In [ ]:
database_url = 'http://docs.sqlalchemy.org/en/latest/core/engines.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(database_url)
HTML(iframe)

In [ ]:
from sqlalchemy import create_engine

# create database tables
engine = create_engine(TEST_DB_PATH)
Base.metadata.create_all(engine)

In this tutorial we use a SQLite database. We use create_engine() to connect to the database.

#### Create Engine
The return value of create_engine() is an instance of Engine, and it represents the core interface to the database.

#### Lazy Connecting
The Engine, when first returned by create_engine(), has not actually tried to connect to the database yet; that happens only the first time it is asked to perform a task against the database

#### Base.metaData.create_all()
As our SQLite database does not yet have a Sections table present, we can issue a CREATE TABLE statements to the database.

Base.metaData.create_all(), passing in our Engine as a source of database connectivity. We will see that special commands are first emitted to check for the presence of the users table, and following that the actual CREATE TABLE statement

In [ ]:
from sqlalchemy.orm import sessionmaker

engine = create_engine(TEST_DB_PATH)
Session = sessionmaker(bind=engine)
session = Session()

#### SQLAlchemy Session

In the most general sense, the Session establishes all conversations with the database and represents a “holding zone” for all the objects which you’ve loaded or associated with it during its lifespan. It provides the entrypoint to acquire a Query object, which sends queries to the database using the Session object’s current database connection, populating result rows into objects that are then stored in the Session

All changes to objects maintained by a Session are tracked - before the database is queried again or before the current transaction is committed, it flushes all pending changes to the database. This is known as the Unit of Work pattern.

#### Insert data into the database

In [ ]:
section = Sections(
      section_name = 'first section'
    , section_text = 'text from the first section'
)

session.add(section)

At this point, we say that the instance is pending; no SQL has yet been issued and the object is not yet represented by a row in the database. 

The Session will issue the SQL to persist the section as soon as is needed, using a process known as a flush. If we query the database for the section, all pending information will first be flushed, and the query is issued immediately thereafter.

In [ ]:
# commit (save) all annual reports to the database
session.commit()

#### Commit

We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit().

commit() flushes the remaining changes to the database, and commits the transaction. The connection resources referenced by the session are now returned to the connection pool. Subsequent operations with this session will occur in a new transaction, which will again re-acquire connection resources when first needed.

In [ ]:
# query database to check if data was added
result = session.query(Sections).all()
for row in result:
    print('section_id: {}'.format(row.section_id))
    print('section_name: {}'.format(row.section_name))
    print('section_text: {}'.format(row.section_text))

In [ ]:
# view the __repr__ of the SQLAlchemy class
result

In [1]:
# Uncomment to delete data from Sections to reset table (optional)

#session.execute("DELETE FROM Sections")
session.commit()

NameError: name 'session' is not defined

## Exercise

Create and query a database containing StackOverflow questions data.

The questions are randomized and contains a mix of verbose text sites as well as sites related to math and programming. The number of questions from each site may vary, and no filtering has been performed on the questions (such as closed questions).

#### Data Dictionary:
- Id - Unique identifier for each question
- Title - The question's title
- Body - The body of the question
- Tags - The tags associated with the question (all lowercase, should not contain tabs '\t' or ampersands '&')

Source: https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction

### Create a table named StackOverflow with the following columns and data types
- tablename = 'STACKOVERFLOW'
- Id - Integer, primary_key
- Title - Text, nullble
- Body - Text, nullable
- Tags - Text, nullable
- Add a __repr__ that prints out the Id and Title

In [ ]:
# Define table here


In [ ]:
# confirm the path to the stackoverflow database
DB_PATH

In [ ]:
# create a engine session to connect to the database 


# SQL

In [ ]:
database_url = 'http://docs.sqlalchemy.org/en/latest/orm/tutorial.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(database_url)
HTML(iframe)

In [ ]:
# SQLAlchamy Core - SQL Expression Language
from sqlalchemy.sql import text

query = text('SELECT * FROM StackOverflow LIMIT 3')
for result in session.execute(query):
    print('Question Id: {}'.format(result.Id))
    print('TITLE: {}'.format(result.Title))
    print('TAGS: {} \n'.format(result.Tags))

In [ ]:
# SQLAlchemy ORM
for result in session.query(StackOverflow)[0:3]:
    print('Question Id: {}'.format(result.Id))
    print('TITLE: {}'.format(result.Title))
    print('TAGS: {} \n'.format(result.Tags))

In [ ]:
# view __repr__ of result
print(session.query(StackOverflow)[0])

In [ ]:
# Pandas
import pandas as pd
pd.read_sql('SELECT * FROM STACKOVERFLOW LIMIT 3', con=engine)

### SELECT

In [ ]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Id, Tags FROM StackOverflow LIMIT 3')
session.execute(query).fetchall()

In [ ]:
# SQLAlchemy ORM
session.query(StackOverflow.Id, StackOverflow.Tags).limit(3).all()

In [ ]:
# Raw SQL Query
print(session.query(StackOverflow.Id, StackOverflow.Tags).limit(3))

In [ ]:
#Pandas 
pd.read_sql('SELECT Id, Tags FROM STACKOVERFLOW LIMIT 3', con=engine)

### WHERE

In [ ]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Id, Tags, Title FROM STACKOVERFLOW WHERE Tags = "python" LIMIT 3')
session.execute(query).fetchall()

In [ ]:
# SQLAlchemy ORM
session.query(StackOverflow.Id, StackOverflow.Title
    ).filter(StackOverflow.Tags == 'python').limit(3).all()

In [ ]:
# Raw SQL Query
print(session.query(StackOverflow.Id, StackOverflow.Title
    ).filter(StackOverflow.Tags == 'python').limit(3))

In [ ]:
# Pandas
pd.read_sql('SELECT Id, Tags, Title FROM STACKOVERFLOW WHERE Tags = "python" LIMIT 3', con=engine)

### LIKE

In [ ]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Tags FROM STACKOVERFLOW WHERE Tags LIKE "%python%" LIMIT 3')
session.execute(query).fetchall()

In [ ]:
# SQLAlchemy ORM
session.query(StackOverflow.Tags
    ).filter(StackOverflow.Tags.like('%python%')).limit(3).all()

In [ ]:
# Raw SQL Query
print(session.query(StackOverflow.Tags
    ).filter(StackOverflow.Tags.like('%python%')).limit(3))

In [ ]:
# Raw SQL Query
print(session.query(StackOverflow.Tags
    ).filter(StackOverflow.Tags.like('%python%')).limit(3))

In [ ]:
# Pandas
pd.read_sql('SELECT Tags FROM STACKOVERFLOW WHERE Tags LIKE "%python%" LIMIT 3', con=engine)

### IN

In [ ]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Title FROM STACKOVERFLOW WHERE Tags IN ("python","java","sql") LIMIT 3')
session.execute(query).fetchall()

In [ ]:
# SQLAlchemy ORM
session.query(StackOverflow.Title
    ).filter(StackOverflow.Tags.in_(['python', 'java', 'sql'])).limit(3).all()

In [ ]:
# Raw SQL Query
print(session.query(StackOverflow.Title
    ).filter(StackOverflow.Tags.in_(['python', 'java', 'sql'])).limit(3))

In [ ]:
# Pandas
pd.read_sql('SELECT Title FROM STACKOVERFLOW WHERE Tags IN ("python","java","sql") LIMIT 3'
            , con=engine)

### COUNT

In [ ]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT COUNT(*) FROM STACKOVERFLOW')
session.execute(query).scalar()  # scalar returns a single value (no tuple)

In [ ]:
# SQLAlchemy ORM
session.query(StackOverflow).count()

In [ ]:
# Pandas
pd.read_sql('SELECT COUNT(*) FROM STACKOVERFLOW', con=engine)

### GROUP BY

In [ ]:
# SQLAlchamy Core - SQL Expression Language
query = text("""SELECT Tags, COUNT(Tags)
            FROM STACKOVERFLOW
            GROUP BY Tags
            ORDER BY COUNT(Tags) Desc
            LIMIT 3""")
session.execute(query).fetchall()  # scalar returns a single value (no tuple)

In [ ]:
# SQLAlchemy ORM
from sqlalchemy import func

session.query(StackOverflow.Tags, func.count(StackOverflow.Tags)
    ).group_by(StackOverflow.Tags
    ).order_by(func.count(StackOverflow.Tags).desc()
    ).limit(3).all()

In [ ]:
# Raw SQL Query
print(session.query(StackOverflow.Tags, func.count(StackOverflow.Tags)
    ).group_by(StackOverflow.Tags
    ).order_by(func.count(StackOverflow.Tags).desc()
    ).limit(3))

In [ ]:
# Pandas
pd.read_sql("""SELECT Tags, COUNT(Tags)
            FROM STACKOVERFLOW
            GROUP BY Tags
            ORDER BY COUNT(Tags) Desc
            LIMIT 3
            """, con=engine)

#### Extra Material - load database iteratively from pandas

In [ ]:
# confirm dataset is available
print('ZIP NAME: {} \n'.format(STACKOVERFLOW_ZIP_NAME))
print('CSV NAME: {} \n'.format(STACKOVERFLOW_CSV_NAME))
print('DB PATH: {}'.format(DB_PATH))

In [ ]:
# NOTE: you must download the stackoverflow zip (2GB zip file) for below code to work
# https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/data

import pandas as pd
import datetime as dt
import zipfile
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

zf = zipfile.ZipFile(STACKOVERFLOW_ZIP_NAME)
engine = create_engine(DB_PATH)
for ind, df in enumerate(pd.read_csv(
    zf.open(STACKOVERFLOW_CSV_NAME)
        , chunksize=10_000
        , iterator=True
        , encoding='utf-8'
        , nrows=50_000)):
    
    # uncomment to write to db 
    #df.to_sql('STACKOVERFLOW', con=engine, if_exists='append')